### combine all features of docuentation of web scrapping
step1 : driver main url of housing.com

step 2 : i will get list of area in form of list

step 3: search for the particular area in housing.com

step4 : scrape data from each card by using switch tabs and closing them 

step 5 : once desired number of data is scraped we are going to return to home page and repeat the process



In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import pandas as pd
chrome_options = Options()
chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
driver.get("https://housing.com/")

In [10]:
chennai_localities = [
    "Royapettah, Chennai",
    "Egmore, Chennai",
    "Besant Nagar, Chennai",
    "Ashok Nagar, Chennai",
    "Medavakkam, Chennai",
    "Chromepet, Chennai",
    "Manapakkam, Chennai",
    "Madipakkam, Chennai"
]

### searching each and every locality to get data 
WE CAN USE BACK METHOD TO GET BACK TO MAIN PAGE

In [11]:
data_of_all_property = []

def get_text(xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except:
        return "N/A"

def safe_print(label, xpath):
    try:
        property_data[label] = get_text(xpath)  # ← store in dict
    except Exception as e:
        property_data[label] = "N/A"

def get_by_label_th(label):
    try:
        xpath = f"//tr[th[contains(text(),'{label}')]]/td/div"
        return driver.find_element(By.XPATH, xpath).text
    except:
        return "N/A"

for location in chennai_localities:
    try:
        search_bar = driver.find_element("xpath",'/html/body/div[1]/div[1]/div[4]/div/div/div[1]/div/div[2]/input')
        search_bar.send_keys(location)
        time.sleep(2)
        search_bar.send_keys(Keys.RETURN)
        time.sleep(2)
        property_filter = driver.find_element("xpath","/html/body/div[1]/div[1]/div[3]/div[1]/div/div[1]")
        property_filter.click()
        try:
            independent_house = driver.find_element("xpath","/html/body/div[1]/div[1]/div[3]/div[1]/div/div[1]/div[2]/div/div/div/div[2]/span/a")
            independent_house.click()
        except :
            print("failed")
        # driver url is updated now like https://housing.com/in/buy/chennai/kodungaiyur_west
        # we can start fetch data from property cards
        time.sleep(4)
        cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'T_topSection')]")
        print(cards)
        # Limit to 20 cards if more exist, otherwise take all
        max_cards = 7
        num_cards_to_process = min(len(cards), max_cards)

        for no_of_card in range(num_cards_to_process):
            # entering into url of seprate property
            time.sleep(4)
            property_page = driver.current_window_handle
            try:
                property_link = driver.find_element("xpath",f"/html/body/div[1]/div[1]/div[3]/div[2]/div[1]/div[4]/div[{no_of_card}]/div/article/div[1]/div[2]/div[1]/div[1]/div[1]/a")
            except:
                continue
            time.sleep(3)
            property_link.click()

            # keeping track of all tab to ensure smooth transmission
            all_tabs_browser = driver.window_handles
            #selecting tab that is not main one
            property_details_tab = [tab for tab in all_tabs_browser if tab!=property_page][0]
            driver.switch_to.window(property_details_tab)

            #### ---------------------------------------- actually fetch part ------------------------------------------
            property_data={}
            property_data["Title"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/h1/div[1]/div")
            property_data["areaname"]=get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/h1/div[2]")
            property_data["Price"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/div[2]/div/div[2]/div[1]/span")
            property_data["Area"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[1]/div[1]/div/div")
            property_data["Avg Price"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[2]/div[1]")
            property_data["Facing"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[4]/div[1]")
            property_data["Furnishing"] = get_text("/html/body/div[1]/div[1]/div[3]/div[1]/div[1]/div/section/div[5]/div[1]")

            property_data["Bedrooms"] = get_by_label_th("Bedrooms")
            property_data["Bathrooms"] = get_by_label_th("Bathrooms")
            property_data["Balcony"] = get_by_label_th("Balcony")

            # Handle score containers
            containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'T_featureContainerStyle')]")
            for container in containers:
                try:
                    label = container.find_element(By.TAG_NAME, "span").text
                    score = container.find_element(By.TAG_NAME, "p").text
                    property_data[label] = score
                except:
                    continue
            #append the new data
            data_of_all_property.append(property_data)
            driver.close()
            driver.switch_to.window(property_page)
        #for bringing the property page to home again
        driver.back()
        driver.back()
    except:
        continue
driver.quit()


[<selenium.webdriver.remote.webelement.WebElement (session="074471e6043d3904877a75385744b945", element="f.4F9CDD8D6C2BC51A1381D8B786A12FB3.d.5D48F5ABC80D19A3EA781F40EE001BBB.e.24611")>, <selenium.webdriver.remote.webelement.WebElement (session="074471e6043d3904877a75385744b945", element="f.4F9CDD8D6C2BC51A1381D8B786A12FB3.d.5D48F5ABC80D19A3EA781F40EE001BBB.e.24711")>, <selenium.webdriver.remote.webelement.WebElement (session="074471e6043d3904877a75385744b945", element="f.4F9CDD8D6C2BC51A1381D8B786A12FB3.d.5D48F5ABC80D19A3EA781F40EE001BBB.e.24786")>, <selenium.webdriver.remote.webelement.WebElement (session="074471e6043d3904877a75385744b945", element="f.4F9CDD8D6C2BC51A1381D8B786A12FB3.d.5D48F5ABC80D19A3EA781F40EE001BBB.e.24866")>, <selenium.webdriver.remote.webelement.WebElement (session="074471e6043d3904877a75385744b945", element="f.4F9CDD8D6C2BC51A1381D8B786A12FB3.d.5D48F5ABC80D19A3EA781F40EE001BBB.e.24918")>, <selenium.webdriver.remote.webelement.WebElement (session="074471e6043d390

## convert it into csv file 



In [12]:
## convert it into csv file 

df = pd.DataFrame(data_of_all_property)
df.to_csv("propert_data2.csv",mode="a",index=False,header=False)
print("data stored sucessfully")

data stored sucessfully
